The images created by diffusers CNXS for **SD2.1** and canny input are clearly not right. They are far too brown. There were 3 bugs, which I've fixed no (see `Compare intermediate results -- 14.ipynb`).

Let's look if the intermediate results on subblock level also look good.

In [1]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'  # needed to make torch deterministic

In [2]:
import torch
from torch.testing import assert_close
from torch import allclose, nn, tensor
torch.set_printoptions(linewidth=200, precision=3, sci_mode=False)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'mps'
device_dtype = torch.float16 if device == 'cuda' else torch.float32

## Load logs

In [4]:
from diffusers.umer_debug_logger import UmerDebugLogger

Info: `UmerDebugLogger` created. This is a logging class that will be deleted when the PR to integrate ControlNet-XS is done.


In [30]:
cloud_cuda = UmerDebugLogger.load_log_objects_from_dir('logs/cloud')
local_cuda = UmerDebugLogger.load_log_objects_from_dir('logs/local_cuda')

print(len(cloud_cuda), len(local_cuda))

for i, (c,l) in enumerate(zip(cloud_cuda, local_cuda)):
    if c.msg!=l.msg: print(f'{i:<3}{c.msg:>20}{l.msg:>20}')

191 191


## Compare intermediate results

In [31]:
def mae(t1,t2):
    assert t1.shape==t2.shape
    return (t1-t2).abs().mean()

In [35]:
from functools import partial
from util_inspect import fmt_bool

def compare_intermediate_results(n=None,n_start=0,prec=5, compare_prec=3, ignore_base=False):
    if n is None: n=max(len(cloud_cuda), len(local_cuda))

    print(f'{"":<3} | {"block":<30} | {"shape":<20} | {"same names?":<12} | {"same shapes?":<12} | {"same values?":<12} | {"Δ cuda local -> cloud":<20}')
    print(f'{"":<3} | {"":<30} | {"":<20} | {"":<12} | {"":<12} | {"prec="+str(compare_prec):^12} | {"prec="+str(prec):^20}')

    def calc_total_len(lens): return sum(lens)+3*len(lens)-1
    total_len = calc_total_len((3,30,20,12,12,12,20))

    line = partial(
        lambda txt, width: print(txt * (width//len(txt))),
        width=total_len
    )
    
    labels = []
    def add_label(lbs, ctrl=True):
        if not isinstance(lbs, (list, tuple)): lbs = [lbs]
        for l in lbs:
            labels.append(('Base',l))
        for l in lbs: 
            if ctrl: labels.append(('Ctrl',l))
    
    # # to separate logs into sections
    lines_at, fat_lines_at = [],[]
    def add_line_after(x): lines_at.append(lines_at[-1]+x)
    def add_fat_line(): fat_lines_at.append(lines_at[-1])
    
    lines_at = [5]                                     # input 
    add_fat_line()
    add_line_after(11)                                 # conv in
    add_fat_line()
    for _ in range(11): add_line_after(13)             # enc       (RA,RA,D / RA,RA,D / RA,RA,D / R,R )
    add_fat_line()
    add_line_after(8)                                  # mid
    add_fat_line()
    for _ in range(12): add_line_after(3)              # dec       (R,R,RU / RA,RA,RAU / RA,RA,RAU / RA,RA,RA )
    add_fat_line()

    # # to describe each log line
    descrs = ['x', 'time info', 'text info', 'guidance image', 'guidance image (projected)']                              # input 
    descrs += [                                                                                                           # conv in
        'base conv in',
        'ctrl conv in',
        'add guided hint to ctrl',
        'ctrl -> base: input', 'ctrl -> base: weight', 'ctrl -> base: bias', 'ctrl -> base: output pre mult', 'ctrl -> base: output', 'add ctrl->base', 
        'output base', 
        'output ctrl'
    ]
    for _ in range(11):descrs += [                                                                                        # enc
        'base -> ctrl: input', 'base -> ctrl: weight', 'base -> ctrl: bias', 'base -> ctrl: output', 'concat base -> ctrl',
        'apply base block',
        'apply ctrl block',
        'ctrl -> base: input', 'ctrl -> base: weight', 'ctrl -> base: bias', 'ctrl -> base: output pre mult', 'ctrl -> base: output', 'add ctrl -> base',
    ]
    descrs += [                                                                                                           # mid
        'enc.connection.input', 'enc.connection.param.weight', 'enc.connection.param.bias', 'enc.connection.output', 'concat base -> ctrl', 
        'apply base block',
        'apply ctrl block',
        'add ctrl->base'
    ]
    for _ in range(12): descrs += ['add ctrl encoder->base decoder','apply base block']                                   # dec 
    descrs += ['base conv out'] 
    
    line('#')
    for i in range(n_start,n):
        cc,lc = cloud_cuda[i], local_cuda[i]
                
        eq_name = cc.msg==lc.msg
        eq_shape = cc.shape==lc.shape
        
        if eq_shape:
            eq_vals = torch.allclose(cc.t,lc.t,atol=10**-compare_prec)
            mae_2 = mae(lc.t,cc.t) 
            mae_2 = ("{:>20."+str(prec)+"f}").format(mae_2)
        else:
            eq_vals,mae_2=False,'inf'
        
        print(f'{i+1:<3} | {cc.msg:<30} | {cc.shape:>20} | {fmt_bool(eq_name, "^12")} | {fmt_bool(eq_shape, "^12")} | {fmt_bool(eq_vals, "^12")} | {mae_2}\t{descrs[i]}')

        if i+1 in fat_lines_at: line('=')
        elif i+1 in lines_at: line('-')

In [36]:
compare_intermediate_results(compare_prec=3, prec=3, ignore_base=False)

    | block                          | shape                | same names?  | same shapes? | same values? | Δ cuda local -> cloud
    |                                |                      |              |              |    prec=3    |        prec=3       
#################################################################################################################################
1   | prep.x                         |       [2, 4, 64, 64] |      y       |      y       |      y       |                0.000	x
2   | prep.temb                      |            [2, 1280] |      y       |      y       |      y       |                0.000	time info
3   | prep.context                   |        [2, 77, 1024] |      y       |      y       |      y       |                0.000	text info
4   | prep.raw_hint                  |     [2, 3, 512, 512] |      y       |      y       |      y       |                0.000	guidance image
5   | prep.guided_hint               |       [2, 4, 64, 64] |   

There are no errors! :)

But still, the created picture is too brown. So let's compare the ouputs after each step - see in `Compare intermediate results -- 16.ipynb`